# Chat server

In [ ]:
//| export

import type { Database } from "tinychat/db.ts";
import { NewServerRecord } from "tinychat/firehose.ts";
import { seedMessages } from "tinychat/core/messaging.ts";

export class Servers {
  constructor(protected db: Database) {}

  createServer(server: NewServerRecord) {
    const createServer = this.db.prepare(`
        INSERT INTO servers (uri, name, creator) VALUES (
          :uri, :name, :creator
        )`);
    const createMembership = this.db.prepare(
      `INSERT INTO server_memberships (user, server) VALUES (
          :creator, :server
        ) ON CONFLICT(user, server) DO NOTHING`,
    );
    const createChannel = this.db.prepare(
      `INSERT INTO channels (id, name, server) VALUES (:id, :name, :server) ON CONFLICT(id, server) DO NOTHING`,
    );

    this.db.transaction(() => {
      createServer.run({
        uri: server.uri,
        name: server.commit.record.name,
        creator: server.did,
      });
      createMembership.run({ creator: server.did, server: server.uri });
      for (const channel of server.commit.record.channels) {
        createChannel.run({
          id: channel.id,
          name: channel.name,
          server: server.uri,
        });
      }
    })();

    // seed messages if needed
    if (Deno.env.get("SEED_MESSAGES_AFTER_SERVER_CREATION")) {
      seedMessages({ db: this.db, server: server.uri });
    }
  }
}

[Module: null prototype] {
  Messaging: [class Messaging],
  seedMessages: [Function: seedMessages]
}